## Bigram Base Model

- A simple model for **baseline**
- This model **predicts next character based on the previous character**

---

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# hyperparameters
batch_size = 32  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 300
learning_rate = 1e-2
eval_iters = 200

device = "cuda" if torch.cuda.is_available() else "cpu"
if(device == "cpu"):
    device = "mps" if torch.backends.mps.is_available() else "cpu"

print(f"using {device=}")

using device='mps'


In [3]:
# ------------

torch.manual_seed(1337) # for reproducibility

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open("./dataset/input.txt", "r", encoding="utf-8") as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]  # encoder: take a string, output a list of integers
decode = lambda l: "".join( [itos[i] for i in l])  # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

train_data[:10]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])

In [4]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # choose random start indices for the batch
    x = torch.stack([data[i : i + block_size] for i in ix]) # pick contiguous chunks of text as the input
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix]) # pick the same chunks, offset by 1, as the target
    x, y = x.to(device), y.to(device) # move the data to the proper device (CPU or GPU)
    return x, y

# sample a batch of data
xb, yb = get_batch("train")

print(xb.shape, yb.shape)  # expect [batch_size x block_size]
print(xb[0], yb[0]) # the first sequence in the batch

torch.Size([32, 8]) torch.Size([32, 8])
tensor([53, 59,  6,  1, 58, 56, 47, 40], device='mps:0') tensor([59,  6,  1, 58, 56, 47, 40, 59], device='mps:0')


In [5]:

@torch.no_grad() # let's make sure to turn off autograd for the sampling, because we will this call in a context where we don't want to backpropagate gradients (for logging and visualization after end of some epochs)
def estimate_loss():
    """_summary_
    Get the loss for the model on the train and val sets
    """
    out = {}
    model.eval() # disable dropout, etc.
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # re-enable dropout, etc.
    return out

In [6]:

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)  # (B,T,C) - Batch, tokens, channels (embedding size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T) # we could have also used the -1 argument to the view function
            loss = F.cross_entropy(logits, targets) # we reshape the logits to be a 2D tensor and the targets to be 1D for the loss function as it expects **(N, C) and (N, 1)** respectively

        return logits, loss

    def generate(self, idx, max_new_tokens):
        """_summary_
            Use only the last token in the context to generate new tokens
        """
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [7]:

model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while (after every 300 iters) evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(
            f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
        )

    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print("training complete")

/Users/Deependu/Library/Python/3.9/lib/python/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


step 0: train loss 4.7687, val loss 4.7648
step 300: train loss 2.8323, val loss 2.8430
step 600: train loss 2.5424, val loss 2.5780
step 900: train loss 2.4902, val loss 2.5132
step 1200: train loss 2.4762, val loss 2.5049
step 1500: train loss 2.4771, val loss 2.4935
step 1800: train loss 2.4801, val loss 2.4951
step 2100: train loss 2.4702, val loss 2.4886
step 2400: train loss 2.4695, val loss 2.4880
step 2700: train loss 2.4693, val loss 2.4893
step 3000: train loss 2.4551, val loss 2.4895
step 3300: train loss 2.4587, val loss 2.4849
step 3600: train loss 2.4525, val loss 2.4924
step 3900: train loss 2.4637, val loss 2.5001
step 4200: train loss 2.4555, val loss 2.4907
step 4500: train loss 2.4590, val loss 2.4817
step 4800: train loss 2.4622, val loss 2.4838
step 5100: train loss 2.4612, val loss 2.4925
step 5400: train loss 2.4648, val loss 2.4958
step 5700: train loss 2.4516, val loss 2.4791
step 6000: train loss 2.4599, val loss 2.4734
step 6300: train loss 2.4633, val loss 2

In [8]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


Foasth prse tize herst el
O u frnie hy:


Hak, CORineg agnthe t rr Masearor charnge?
Ten, re thy, chouspo in mpery way avend ouburser sickes bekecard dhiceny

He tw el fe oupise he, lbustselownthous;
Nom w
T:
The at;
I me Ed fris, g he itheland's oe, oghithet f, badogienthofathatey foueay wad,
ureisold array n
ICoyockind m murs, in mamybalorenyongmyooe, d Vofetthindy hak shil brveseay alsteanerm to, oupomp rede d pre h, gackefithrerean ththin, thind my:
Berouerse IOLUEDed nghathicerire.
In IS:
I


---

## Insights

- Most of the output is **non-sense**, but it's a good start for a baseline model
- The model is **simple** and **fast**
- It predicts the next character based on the previous character

---

## Save model

1. Save model's state_dict

In [9]:
torch.save(model.state_dict(), "./model/bigram-model/bigram-model-state-dict.pth")

In [10]:
model.state_dict()

OrderedDict([('token_embedding_table.weight',
              tensor([[  2.1663,  -8.9142,  -7.7765,  ...,  -9.5859,  -2.7291, -10.1730],
                      [ -5.9427,  -4.5666,  -7.7469,  ...,  -9.2596,   0.6270,  -4.7146],
                      [  1.9087,   1.5847, -10.8409,  ...,  -9.4011,  -9.1053,  -8.9093],
                      ...,
                      [ -3.1896,  -0.0140,  -8.8260,  ...,  -6.7737,  -2.1301,  -7.2833],
                      [ -0.2109,   2.7150,  -1.2015,  ...,  -9.5673,  -8.3921,  -9.5525],
                      [ -7.7607,  -1.5148,  -7.2700,  ...,  -6.4779,  -0.6521,  -0.7968]],
                     device='mps:0'))])

---

2. Save model's architecture

In [11]:
torch.save(model, "./model/bigram-model/bigram-model-with-architecture.pth")